# Advanced Topics in ML 

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

#### change directory if neccessary #####################################################################
MODEL_FOLDER   = "C:/Users/Anita/Documents/UCL/AdvancedTopicsML/Assignment_2/models/pixel_prediction_3x32/"
#########################################################################################################

MODEL_FILENAME = MODEL_FOLDER+"rnn_model_gru_3_layer_pixel_prediction.ckpt" 
MODEL_FILENAME_iter = MODEL_FOLDER+"rnn_model_gru_3_layer_pixel_prediction_iter.ckpt"

logs_path = 'trained_models/pixel_prediction_3_layer'

hm_epochs = 40
n_output = 1
batch_size = 128
chunk_size = 1
n_chunks = 784
rnn_size = 32
learning_rate = tf.placeholder('float')
num_runs = int(mnist.train.num_examples/batch_size)

    

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# threshold function
def binarize(images, threshold=0.1):
    return (threshold < images).astype('float32')

<h4>Define model</h4>

In [3]:
# define placeholders
x = tf.placeholder('float', [None, n_chunks,chunk_size],name='InputData')
y = tf.placeholder('float',name='LabelData')

# define Variables for hidden layers
    
layer1 = {'weights':tf.Variable(tf.random_normal([rnn_size,n_output]),name='Weights1'), #32 x 1
         'biases':tf.Variable(tf.random_normal([n_output]),name='Bias1')}

# define rnn cell    
gru_cell = tf.nn.rnn_cell.GRUCell(rnn_size)
gru_cell = tf.nn.rnn_cell.MultiRNNCell(cells=[gru_cell] * 3)
outputs, states = tf.nn.dynamic_rnn(cell = gru_cell, inputs = x,dtype=tf.float32)

# flatten
outputs_reshaped = tf.reshape(outputs, [-1, rnn_size]) #(128*784) x 32

# first hidden layer
output = tf.matmul(outputs_reshaped,layer1['weights']) + layer1['biases'] # (128*784) x 1
prediction = tf.reshape(output,[-1,n_chunks])
prediction = prediction[:,0:783]

groundtruth = tf.reshape(x,[-1,n_chunks])
groundtruth = groundtruth[:,1:]


In [4]:
# define loss function and optimizer

with tf.name_scope('Model'):
    logits = prediction
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits,groundtruth),1))
with tf.name_scope('Optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        
    
# Create summary for loss and accuracy
tf.summary.scalar("loss", loss)
merged_summary_op = tf.summary.merge_all()

#init saver
saver = tf.train.Saver()

<h4>Train model - (!!! Don't rerun this cell !!!)</h4>

In [ ]:
### DO NOT RUN ###
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(hm_epochs):
        if (epoch < 25):
            lr=0.0003
        else:
            lr=0.00005
        for i in range(num_runs): 
            epoch_x, epoch_y = mnist.train.next_batch(batch_size)
            epoch_x = binarize(epoch_x)
            epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))
            _, _ , summary = sess.run([optimizer, loss ,merged_summary_op],
                                           feed_dict={x: epoch_x,y: epoch_y,learning_rate:lr})
            summary_writer.add_summary(summary, epoch * num_runs + i)
        saver.save(sess, MODEL_FILENAME_iter)
    print('Training finished')        
    saver.save(sess, MODEL_FILENAME)
    print('Model saved') 

<h4>Restore model</h4>

In [5]:
# restore model to get testing loss 
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess = sess, save_path= MODEL_FILENAME)
    print("Restored values:")
    epoch_X = mnist.test.images
    epoch_Y = mnist.test.labels
    epoch_X = binarize(epoch_X)
    epoch_X = epoch_X.reshape((-1,n_chunks,chunk_size))    
    loss_test = sess.run(loss,feed_dict={x:epoch_X, y:epoch_Y})
    print('Testing loss:',loss_test) 

Restored values:
Testing loss: 73.0449


In [6]:
# restore model to get training loss
# divide training set in 10 deterministic batches for easier computation
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess = sess, save_path= MODEL_FILENAME)
    print("Restored values:")
    loss_train_sum = 0
    for j in range(10):
        epoch_X = mnist.train.images[5500*j:5500*(j+1),:]
        epoch_Y = mnist.train.labels[5500*j:5500*(j+1),:]
        epoch_X = binarize(epoch_X)
        epoch_X = epoch_X.reshape((-1,n_chunks,chunk_size))    
        loss_train = sess.run(loss,feed_dict={x:epoch_X, y:epoch_Y})
        loss_train_sum += loss_train
    loss_train_ave = loss_train_sum/10
    print('Training loss:',loss_train_ave)

Restored values:
Training loss: 73.6893409729
